In [1]:
import os
import pandas as pd
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import joblib
import dproc, sgutil, sgpp, sgnn, sgml
import warnings

# 모든 FutureWarning 무시
warnings.simplefilter(action='ignore', category=FutureWarning)

2025-05-06 09:41:39.099040: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 09:41:39.128501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746524499.152398     984 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746524499.159357     984 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 09:41:39.184920: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Data Processing

In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, images_test = list(), list()
    spots, spots_test = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = np.array(train_images[slide_name])
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
            if slide_name == 'S_2':
                spots[-1] = spots[-1].assign(
                    x = lambda x: x['x'] - 60,
                    y = lambda x: x['y'] - 60,
                )
        # Test 이미지를 불러옵니다.
        test_images = h5file["images/Test"]
        test_spots = h5file["spots/Test"]
        sample = 'S_7'
        image = np.array(test_images[sample])
        p1 = 2000 - image.shape[0]
        p2 = 2000 - image.shape[1]
        images_test.append(np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge'))
        spots_test.append(pd.DataFrame(np.array(test_spots[sample])).assign(slide = 0))
        df_spots = pd.concat(spots).reset_index(drop = True)
        images = np.stack(images)
        images_test = np.stack(images_test)
        df_spots_test = pd.concat(spots_test).reset_index(drop = True)
    return images, df_spots, images_test, df_spots_test

def make_img_proc_info(df, img_width, img_height):
    return df.assign(
        left = lambda x: (x['x'] - img_width // 2).astype('int'),
        right = lambda x: (x['left'] + img_width).astype('int'),
        top = lambda x: (x['y'] - img_height // 2).astype('int'),
        bottom = lambda x: (x['top'] + img_height).astype('int'),
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )


def create_df(df, img_width, img_height):
    df = make_img_proc_info(df, img_width, img_height)
    df_pixel = df[['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']].apply(
        lambda x: pd.Series(proc_images_np(x, images)), axis = 1
    ).rename(columns = lambda x: 'pixel_{}'.format(x)).reset_index(drop = True)
    X_pixel = df_pixel.columns
    return df.join(df_pixel), X_pixel

def proc_images_np(X, images):
    return np.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)], 'edge'
    ).flatten()


images, df_spots, images_test, df_spots_test = load_data("data/elucidata_ai_challenge_data.h5")
targets = [i for i in df_spots.columns if i.startswith('C')]
n_components = 35
targets2 = ['C{}_l'.format(i + 1) for i in range(n_components)]
df_spots= df_spots.join(
    np.log(df_spots[targets]).rename(columns = lambda x: x + '_l')
)

In [3]:
df_spots = df_spots.join(
    df_spots.groupby('slide')[['x', 'y']].transform(
        lambda x: (x - x.min()) * 2 / (x.max() - x.min()) - 1
    ).rename(columns = lambda x: x +'_mm')
)

In [4]:
images_tf = tf.constant(images)
images_test_tf = tf.constant(images_test)

2025-05-06 09:42:03.444776: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1746524523.444874     984 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
from scipy.stats import spearmanr
import xgboost as xgb
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, train_test_split

gkf = GroupKFold(6)
ss = GroupShuffleSplit(1)
sc = sgutil.SGCache('img', 'result', 'model')

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

def spearman(df, df_prds):
    return df_prds.apply(
        lambda x: spearmanr(x, df.loc[x.name, targets])[0],axis=1
    ).mean()

config = {
    'predict_func': lambda m, df, X: pd.DataFrame(np.exp(m.predict(df[X])), index = df.index, columns = targets2),
    'score_func': lambda df, prds: spearman(df[targets], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': targets2, 'groups': 'slide'
}


xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor, progress = 50)
lasso_adapter = sgml.SklearnAdapter(Lasso)
ridge_adapter = sgml.SklearnAdapter(Ridge)
mlp_adapter = sgml.SklearnAdapter(MLPRegressor)

lr_adapter = sgml.SklearnAdapter(LinearRegression)

In [5]:
def load_encoder(filename):
    enet_, d_ = joblib.load(filename)
    input_shape = (226, 226, 3)
    enet = tf.keras.applications.EfficientNetB0(
        include_top = False, 
        weights = None,
        input_shape = input_shape,
        pooling = 'avg'
    )
    d = tf.keras.layers.Dense(128, activation='swish')
        
    inputs = tf.keras.Input(shape = input_shape)
    x = enet(inputs)
    enet.trainable = True 
    x = d(x)
    m = tf.keras.models.Model(inputs=inputs, outputs=x, name='encoder')
    enet.set_weights(enet_)
    d.set_weights(d_)
    return m

def proc_images(X, images):
    return tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1.0
    )

def create_tf_ds(df):
    return tf.data.Dataset.from_tensor_slices({
        i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
    })

In [6]:
encoder = load_encoder('model/ae_encoder_b0_obj.joblib')
ds_ae = create_tf_ds(
    make_img_proc_info(df_spots, 226, 226)
).map(
    lambda X: proc_images(X, images_tf)
).batch(32)

In [7]:
df_b0 = pd.DataFrame(
    encoder.predict(ds_ae), index = df_spots.index
).rename(columns = lambda x: 'b0_{}'.format(x))
del encoder, ds_ae

I0000 00:00:1746524547.959432    1043 service.cc:148] XLA service 0x7f90fc002d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746524547.959503    1043 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-05-06 09:42:28.022318: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746524548.356080    1043 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-06 09:42:28.721819: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2025_0', 112 bytes spill stores, 144 bytes spill loads

2025-05-06 09:42:28.772217: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_

 10/261 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step

I0000 00:00:1746524555.610641    1043 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


259/261 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

2025-05-06 09:42:43.567442: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2023_0', 344 bytes spill stores, 520 bytes spill loads

2025-05-06 09:42:43.641379: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2023', 16 bytes spill stores, 16 bytes spill loads

2025-05-06 09:42:43.866452: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2023', 180 bytes spill stores, 180 bytes spill loads



261/261 ━━━━━━━━━━━━━━━━━━━━ 24s 56ms/step


In [9]:
X_b0 = df_b0.columns.tolist()
df_b0.head()

,b0_0,b0_1,b0_2,b0_3,b0_4,b0_5,b0_6,b0_7,b0_8,b0_9,...,b0_118,b0_119,b0_120,b0_121,b0_122,b0_123,b0_124,b0_125,b0_126,b0_127
0,0.140363,-0.238408,0.080734,-0.107481,-0.270410,-0.271883,-0.037892,-0.027653,0.361228,0.000955,...,0.426269,0.267163,0.099898,-0.021706,0.389499,-0.092467,-0.009360,0.071034,-0.038935,-0.254953
1,0.264336,0.247747,0.134867,0.307903,-0.231781,0.208133,-0.076133,0.311000,-0.020295,0.126543,...,0.886439,0.434933,-0.129053,0.265150,0.466025,0.953229,0.109224,0.839465,0.207375,-0.270280
2,0.356774,-0.199065,0.070186,-0.180848,-0.277369,-0.276725,0.003389,-0.011188,0.513300,0.324312,...,0.201785,0.252915,0.259053,0.012251,0.238216,-0.202938,0.076162,0.030758,-0.075219,-0.246049
3,0.718715,-0.119620,0.475300,0.023888,-0.244034,-0.017857,0.619920,0.324761,0.788967,0.739314,...,0.542703,0.627571,1.010669,-0.070838,0.968793,-0.132555,0.503486,0.163914,-0.081369,-0.224730
4,0.767667,-0.054608,0.380651,-0.001071,-0.215222,0.010089,0.047771,0.322129,0.532374,0.744368,...,0.827150,0.379626,0.220098,0.148451,1.045769,0.050113,0.428615,0.641775,-0.031105,-0.235014


In [104]:
df_spots8, X_pixel8 = create_df(df_spots, 8, 8)
df_spots9, X_pixel9 = create_df(df_spots, 9, 9)

In [76]:
df_spots_b0 = df_spots.join(df_b0)

# Linear Regression1

In [5]:
result_lr = sc.cv_result(
    'LR', df_spots8, gkf, {'X_num': X_pixel8.tolist()}, config, lr_adapter, result_proc = [sgml.lr_learning_result]
)
np.mean(result_lr['valid_scores']), result_lr['valid_scores']

(0.45159830844446774,
 [0.5297622132553462,
  0.35523506124946186,
  0.285001394714117,
  0.4690515127702302,
  0.5735355011569846,
  0.4970041675206669])

#  XGBoost1

In [6]:
hparams = {
    'model_params': {'max_depth': 2, 'learning_rate': 0.01, 'n_estimators': 100},
    'X_num': X_pixel8.tolist()
}
result_xgb = sc.cv_result('xgb', df_spots8, gkf, hparams, config, xgb_adapter)
np.mean(result_xgb['valid_scores']), result_xgb['valid_scores']

(0.45333574818398664,
 [0.5581417589062058,
  0.3278171532609403,
  0.2877363706365074,
  0.44514948355079187,
  0.5775053789631795,
  0.523664343786295])

# Linear Regression 2

- Linear Regression + PCA

In [7]:
hparams = {'pca': {'X_num': X_pixel8.tolist(), 'hparams': {'n_components': 0.7}}}
result_lr2 = sc.cv_result(
    'LR2', df_spots8, gkf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result]
)
np.mean(result_lr2['valid_scores']), result_lr2['valid_scores']

(0.4785741677274913,
 [0.585518577475043,
  0.3823903667829625,
  0.2811122302230374,
  0.47779563383904533,
  0.6224199244915357,
  0.5222082735533239])

# Linear Regression 3

- Linear Regression + PCA + pixel9

In [8]:
hparams = {'pca': {'X_num': X_pixel9.tolist(), 'hparams': {'n_components': 0.5}}}
result_lr3 = sc.cv_result(
    'LR3', df_spots9, gkf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result], rerun = 0
)
np.mean(result_lr3['valid_scores']), result_lr3['valid_scores']

(0.4788232912064534,
 [0.5861936845222671,
  0.38231418193130695,
  0.283400730596353,
  0.47704874704726036,
  0.6214675435391548,
  0.5225148596023776])

# Ridge1

In [9]:
hparams = {'X_num': X_pixel8.tolist(), 'model_params': {'alpha': 1e7}}
result_rd1 = sc.cv_result(
    'Ridge1', df_spots8, gkf, hparams, config, ridge_adapter, result_proc = [sgml.lr_learning_result], rerun = 0
)
np.mean(result_rd1['valid_scores']), result_rd1['valid_scores']

(0.4953370298393036,
 [0.5607180493342712,
  0.44186317221472116,
  0.27515070428887123,
  0.4832064923694837,
  0.6716981285267731,
  0.5393856323017012])

# Lasso1

In [10]:
hparams = {'X_num': X_pixel8.tolist(), 'model_params': {'alpha': 10}}
result_lasso1 = sc.cv_result(
    'lasso1', df_spots8, gkf, hparams, config, lasso_adapter, result_proc = [sgml.lr_learning_result], rerun = 0
)
np.mean(result_lasso1['valid_scores']), result_lasso1['valid_scores']

(0.49562809082597,
 [0.5607180493342712,
  0.44186317221472116,
  0.27515070428887123,
  0.4832064923694837,
  0.6734624284496407,
  0.5393676982988318])

# Lasso2

In [83]:
hparams = {'X_num': X_b0, 'model_params': {'alpha': 10}}
result_lasso2 = sc.cv_result(
    'lasso2', df_spots_b0, gkf, hparams, config, lasso_adapter, result_proc = [sgml.lr_learning_result], rerun = 1
)
np.mean(result_lasso2['valid_scores']), result_lasso2['valid_scores']

Fold:   0%|          | 0/6 [00:00<?, ?it/s]

(0.49562809082597,
 [0.5607180493342712,
  0.44186317221472116,
  0.27515070428887123,
  0.4832064923694837,
  0.6734624284496407,
  0.5393676982988318])

In [96]:
result_lasso2['model_result'][0]['coef'].abs().sum().to_frame().T

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Lasso3

In [111]:
hparams = {'X_num': ['x_mm', 'y_mm'], 'model_params': {'alpha': 10}}
result_lasso3 = sc.cv_result(
    'lasso3', df_spots, gkf, hparams, config, lasso_adapter, result_proc = [sgml.lr_learning_result], rerun = 1
)
np.mean(result_lasso3['valid_scores']), result_lasso3['valid_scores']

Fold:   0%|          | 0/6 [00:00<?, ?it/s]

(0.49562809082597,
 [0.5607180493342712,
  0.44186317221472116,
  0.27515070428887123,
  0.4832064923694837,
  0.6734624284496407,
  0.5393676982988318])

In [112]:
sc.read_prd('lasso3')

array([[0.15770092, 0.07055206, 0.13902968, ..., 0.00117523, 0.00358429,
        0.02329139],
       [0.15770092, 0.07055206, 0.13902968, ..., 0.00117523, 0.00358429,
        0.02329139],
       [0.15770092, 0.07055206, 0.13902968, ..., 0.00117523, 0.00358429,
        0.02329139],
       ...,
       [0.11007462, 0.06376665, 0.10373893, ..., 0.00114767, 0.00258734,
        0.02366623],
       [0.11007462, 0.06376665, 0.10373893, ..., 0.00114767, 0.00258734,
        0.02366623],
       [0.11007462, 0.06376665, 0.10373893, ..., 0.00114767, 0.00258734,
        0.02366623]])

# MLP1

In [105]:
hparams = {'X_num': X_pixel8.tolist(), 'model_params': {'alpha': 1e4, 'max_iter': 1000, 'hidden_layer_sizes' : (16, )}}
result_mlp1 = sc.cv_result(
    'mlp1', df_spots8, gkf, hparams, config, mlp_adapter, rerun = 0
)
np.mean(result_mlp1['valid_scores']), result_mlp1['valid_scores']

(0.49370322399374444,
 [0.5401794197855354,
  0.44183474026843333,
  0.27287523238275635,
  0.48265547162420147,
  0.6751305159745059,
  0.5495439639270343])

In [108]:
sc.read_prd('mlp1')

array([[0.15614276, 0.06943421, 0.13809256, ..., 0.00125495, 0.00360528,
        0.02353239],
       [0.15614091, 0.06943325, 0.13809147, ..., 0.00125487, 0.00360519,
        0.0235326 ],
       [0.1561428 , 0.06943423, 0.13809259, ..., 0.00125495, 0.00360529,
        0.02353239],
       ...,
       [0.10782869, 0.06406914, 0.10181706, ..., 0.00115206, 0.00258821,
        0.02326384],
       [0.10783844, 0.06406748, 0.10182519, ..., 0.00115217, 0.00258827,
        0.02326533],
       [0.10782605, 0.0640696 , 0.10181483, ..., 0.00115203, 0.00258819,
        0.02326344]], dtype=float32)

# Ensemble

In [31]:
prd = np.zeros_like(df_spots[targets])
for i in [
    #sc.read_prd('ridge1', df_spots.index, columns = targets),
    sc.read_prd('lasso1', df_spots.index, columns = targets),
    #sc.read_prd('mlp1', df_spots.index, columns = targets),
]:
    prd += i[targets].rank(axis = 1)

spearman(
    df_spots[targets],
    pd.DataFrame(prd, index = df_spots.index, columns = targets)
)

# Train

In [35]:
result = sc.train_cv('lasso1', df_spots8, config)